In [2]:
import os
import json
import psycopg2
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
import folium
import folium.plugins
import fiona
from pyproj import Proj, transform
import osmnx as ox
import networkx as nx
import matplotlib.colors as colors
import matplotlib.cm as cm
from shapely.ops import cascaded_union
import geojson
import operator

In [3]:
nx.__version__

'2.0'

In [4]:
import matplotlib.cm as cmx
import matplotlib.colors as colors

def style_function(feature):
    color = feature['properties']['color']
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': "rgb(0, 0, 255)" if color == 2 else "rgb(255, 0, 0)" 
    }

In [5]:
ox.config(log_file=True, log_console=False, use_cache=True)

In [6]:
location = [47.22662, 8.81834]

In [7]:
tags = {
  "landuse_tags": [
    "retail"
  ],
  "amenity_tags": [
    "pub",
    "bar",
    "cafe",
    "restaurant",
    "pharmacy",
    "bank",
    "fast_food",
    "food_court",
    "ice_cream",
    "library",
    "ferry_terminal",
    "clinic",
    "doctors",
    "hospital",
    "pharmacy",
    "veterinary",
    "dentist",
    "arts_centre",
    "cinema",
    "community_centre",
    "casino",
    "fountain",
    "nightclub",
    "studio",
    "theatre",
    "dojo",
    "internet_cafe",
    "marketplace",
    "post_opffice",
    "townhall"
  ],
  "shop_tags": [
    "mall",
    "bakery",
    "beverages",
    "butcher",
    "chocolate",
    "coffee",
    "confectionery",
    "deli",
    "frozen_food",
    "greengrocer",
    "healthfood",
    "ice_cream",
    "pasta",
    "pastry",
    "seafood",
    "spices",
    "tea",
    "department_store",
    "supermarket",
    "bag",
    "boutique",
    "clothes",
    "fashion",
    "jewelry",
    "leather",
    "shoes",
    "tailor",
    "watches",
    "chemist",
    "cosmetics",
    "hairdresser",
    "medical_supply",
    "electrical",
    "hareware",
    "electronics",
    "sports",
    "swimming_pool",
    "collector",
    "games",
    "music",
    "books",
    "gift",
    "stationery",
    "ticket",
    "laundry",
    "pet",
    "tobacco",
    "toys"
  ],
  "leisure_tags": [
    "adult_gaming_centre",
    "amusement_arcade",
    "beach_resort",
    "fitness_centre",
    "garden",
    "ice_rink",
    "sports_centre",
    "water_park"
  ]
}


In [8]:
class AoiQueries():
    def __init__(self, location=None, tags=[], dbscan_eps=35, dbscan_minpoints=3):
        self.location = location
        self.tags = tags
        self.dbscan_eps = dbscan_eps
        self.dbscan_minpoints = dbscan_minpoints


    def _polygons_query(self):
        return """
(SELECT way AS geometry FROM planet_osm_polygon
    WHERE (amenity = ANY(ARRAY{amenity_tags})
            OR shop = ANY(ARRAY{shop_tags})
            OR leisure = ANY(ARRAY{leisure_tags})
            OR landuse = ANY(ARRAY{landuse_tags}))

           AND access IS DISTINCT FROM 'private'
           AND st_within(way, {bbox}))

UNION ALL

(SELECT polygon.way AS geometry FROM planet_osm_polygon AS polygon
    INNER JOIN planet_osm_point AS point
        ON st_within(point.way, polygon.way)
    WHERE (point.amenity = ANY(ARRAY{amenity_tags})
            OR point.shop = ANY(ARRAY{shop_tags})
            OR point.leisure = ANY(ARRAY{leisure_tags})
            OR point.landuse = ANY(ARRAY{landuse_tags}))

        AND point.access IS DISTINCT FROM 'private'
        AND st_within(point.way, {bbox})
        AND polygon.building IS NOT NULL)
        """.format(bbox=self._bbox_query(), **self.tags)

    def _clusters_query(self):
        return """
WITH polygons AS ({polygons_query})
SELECT polygon.geometry AS geometry,
       ST_ClusterDBSCAN(polygon.geometry, eps := {eps}, minpoints := {minpoints}) over () AS cid
FROM polygons AS polygon
        """.format(polygons_query=self._polygons_query(), eps=self.dbscan_eps, minpoints=self.dbscan_minpoints)

    def _hulls_query(self):
        return """
WITH clusters AS ({clusters_query})
SELECT cid, ST_ConvexHull(ST_Union(geometry)) AS geometry
FROM clusters
WHERE cid IS NOT NULL
GROUP BY cid
        """.format(clusters_query=self._clusters_query())

    def _clusters_and_hulls_query(self):
        return """
WITH clusters AS ({clusters_query}),
hulls AS ({hulls_query})
SELECT cid, geometry FROM clusters
UNION ALL
SELECT cid, geometry FROM hulls
        """.format(clusters_query=self._clusters_query(), hulls_query=self._hulls_query())
    
    def _bbox_query(self):
        location_3857 = transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), self.location[1], self.location[0])
        location_3857 = " ".join([str(coordinate) for coordinate in location_3857])

        return """
    (SELECT ST_Buffer(ST_GeomFromText('POINT({})', 3857), 1000) AS bbox)
        """.format(location_3857)



In [9]:
eps = 35
minpts = GREATEST(2, round((-5.342775355 * 10^(-7) * area + 5.738819175 * 10^(-3) * pois_count + 2.912834423)))

hulls_query = AoiQueries(location=location, tags=tags, dbscan_eps=50)._hulls_query()

In [10]:
with psycopg2.connect("") as conn:
    aois = gpd.read_postgis(hulls_query, conn, geom_col='geometry')
    
    aois.crs = fiona.crs.from_epsg(3857)
    
    aois_4326 = aois.to_crs(fiona.crs.from_epsg(4326))


In [44]:
m = folium.Map(location=location, zoom_start=16)
folium.plugins.Fullscreen().add_to(m)

folium.GeoJson(aois).add_to(m)

m

In [33]:
for aoi in aois_4326.geometry:
    aoi_graph = ox.graph_from_polygon(aoi.buffer(0.001), network_type='all')
    ox.plot_graph_folium(aoi_graph, graph_map=m, popup_attribute='name', edge_width=2)
 
m.location = location
m.zoom_start = 16
m

In [42]:
aois_4326 = aois.to_crs(fiona.crs.from_epsg(4326))

all_nodes_centrality = {}
top_20_percent_nodes_centrality = []
for aoi in aois_4326.geometry:
    aoi_graph = ox.graph_from_polygon(aoi.buffer(0.001), network_type='all')
    closeness_centrality = nx.closeness_centrality(aoi_graph)
    all_nodes_centrality = {**all_nodes_centrality, **closeness_centrality}
    
    sorted_nodes = sorted(closeness_centrality.items(), key=operator.itemgetter(1), reverse=True)
    top_20_percent_nodes_centrality += [node[0] for node in sorted_nodes[:len(sorted_nodes) // 10]]

In [45]:
points_query = """
SELECT st_transform(st_geomfromtext('POINT ('||lon/100||' '||lat/100||')',900913), 3857) AS geometry
FROM planet_osm_nodes
WHERE id = ANY(ARRAY[{}])
"""

all_node_ids = ', '.join([f'{key}' for key in all_nodes_centrality.keys()])

query = points_query.format(all_node_ids)
central_points = gpd.read_postgis(query, conn, geom_col='geometry')
central_points.crs = fiona.crs.from_epsg(3857)

m = folium.Map(location=location, zoom_start=16)
folium.plugins.Fullscreen().add_to(m)
#print(nodes_centrality.keys() - list(central_points.osm_id))

if(central_points.size > 0): {
    folium.GeoJson(central_points).add_to(m)
}
        
m

In [46]:
joined_node_ids = ', '.join([f'{key}' for key in top_20_percent_nodes_centrality])

In [47]:
points_query = """
SELECT st_transform(st_geomfromtext('POINT ('||lon/100||' '||lat/100||')',900913), 3857) AS geometry
FROM planet_osm_nodes
WHERE id = ANY(ARRAY[{}])
"""

query = points_query.format(joined_node_ids)
central_points = gpd.read_postgis(query, conn, geom_col='geometry')
central_points.crs = fiona.crs.from_epsg(3857)

m = folium.Map(location=location, zoom_start=16)
folium.plugins.Fullscreen().add_to(m)

if(central_points.size > 0): {
    folium.GeoJson(central_points).add_to(m)
}
        
m
    

In [48]:
ways_query = """
WITH hulls AS ({})
SELECT way AS geometry FROM planet_osm_line, hulls 
WHERE osm_id = ANY(
  SELECT id FROM planet_osm_ways
  WHERE nodes && ARRAY[{}]::bigint[]
)
AND ST_DWithin(hulls.geometry, planet_osm_line.way, 50) 
"""

#m = folium.Map(location=location, zoom_start=16)
#folium.plugins.Fullscreen().add_to(m)

with psycopg2.connect("") as conn:
    query = ways_query.format(hulls_query, joined_node_ids)
    central_points = gpd.read_postgis(query, conn, geom_col='geometry')
    central_points.crs = fiona.crs.from_epsg(3857)

    if(central_points.size > 0): {
        folium.GeoJson(central_points).add_to(m)
    }

m

In [49]:
ways_query = """
WITH hulls AS ({})
SELECT ST_Intersection(way, ST_Buffer(hulls.geometry, 50)) AS geometry FROM planet_osm_line, hulls 
WHERE osm_id = ANY(
  SELECT id FROM planet_osm_ways
  WHERE nodes && ARRAY[{}]::bigint[]
)
AND ST_DWithin(hulls.geometry, planet_osm_line.way, 50) 
"""

#m = folium.Map(location=location, zoom_start=16)
#folium.plugins.Fullscreen().add_to(m)

with psycopg2.connect("") as conn:
    query = ways_query.format(hulls_query, joined_node_ids)
    central_points = gpd.read_postgis(query, conn, geom_col='geometry')
    central_points.crs = fiona.crs.from_epsg(3857)

    if(central_points.size > 0): {
        folium.GeoJson(central_points).add_to(m)
    }

m

In [50]:
ways_query = """
WITH hulls AS ({}),
intersecting_lines AS (
    SELECT hulls.cid, ST_Intersection(way, ST_Buffer(hulls.geometry, 50)) AS geometry FROM planet_osm_line, hulls 
    WHERE osm_id = ANY(
      SELECT id FROM planet_osm_ways
      WHERE nodes && ARRAY[{}]::bigint[]
    )
    AND ST_DWithin(planet_osm_line.way, hulls.geometry, 50)
)

SELECT 1 AS color, ST_ConcaveHull(ST_Union(geometry), 0.99) AS geometry FROM (
  SELECT cid, geometry FROM hulls
  UNION
  SELECT cid, geometry FROM intersecting_lines
) AS tmp
GROUP BY cid

UNION

SELECT 2 as color, geometry FROM hulls
"""


with psycopg2.connect("") as conn:
    query = ways_query.format(hulls_query, joined_node_ids)
    central_points = gpd.read_postgis(query, conn, geom_col='geometry')
    central_points.crs = fiona.crs.from_epsg(3857)

In [51]:
m = folium.Map(location=location, zoom_start=16)

if(central_points.size > 0): {
    folium.GeoJson(central_points, style_function=style_function).add_to(m)
}

m